# Compare the performance of models (loss/accuracy, latency)

The models to compare will  be:
* ResNet18
* ResNet50
* VGG16
* MobileNetV3 - small
* MobileNetV3 - large
* EfficientNet

In [6]:
import torch
from pathlib import Path
from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from VGGs import *
from MobileNetV3 import *
from engine import *

change_to_disk()
data_dir = Path("split_1k/")

In [7]:
data_transform = transforms.Compose([
    # Resize our images to 224x224
    transforms.Resize(size=(224, 224)),
    # Flip the images randomly on the horizontal (just to make it as independent from the position of the pipeline)
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    # Turn the image into a torch.Tensor
    transforms.ToTensor(),
    #Normalize
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

BATCH_SIZE = 32
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size=BATCH_SIZE)

In [8]:
# Things to play with
# Hyperparameters -> NUM_EPOCHS - 100 | Learning_rate - 0.01 
# Functions -> Optimizer - Rprop | Loss Function - Cross Entropy Loss

NUM_EPOCHS = 35
device = get_device()

resnet18 = ResNet18()
resnet50 = ResNet50()
vgg16 = VGG16()
mobnet_sml = MobileNetV3("small")
mobnet_lrg = MobileNetV3("large")

loss_fn = nn.CrossEntropyLoss()

In [9]:
#Train ResNet18
resnet18.to(device)
rprop_optim = torch.optim.Rprop(params=resnet18.parameters(), lr=0.001)
resnet18_results = train(resnet18, train_dl, validation_dl, optimizer=rprop_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

  0%|          | 0/35 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.4552 | train_acc: 0.7970 | validation_loss: 0.3252 | validation_acc: 0.8445
Epoch: 2 | train_loss: 0.3619 | train_acc: 0.8515 | validation_loss: 0.3253 | validation_acc: 0.8523
Epoch: 3 | train_loss: 0.3608 | train_acc: 0.8500 | validation_loss: 0.3091 | validation_acc: 0.8680
Epoch: 4 | train_loss: 0.3568 | train_acc: 0.8535 | validation_loss: 0.3002 | validation_acc: 0.8738
Epoch: 5 | train_loss: 0.3431 | train_acc: 0.8525 | validation_loss: 0.3023 | validation_acc: 0.8641
Epoch: 6 | train_loss: 0.3325 | train_acc: 0.8608 | validation_loss: 0.2907 | validation_acc: 0.8680
Epoch: 7 | train_loss: 0.3391 | train_acc: 0.8535 | validation_loss: 0.2810 | validation_acc: 0.8777


In [ ]:
images, labels, probs = get_predictions(resnet18_results,test_dl,device)
pred_labels = torch.argmax(probs, 1)
plot_confusion_matrix(labels, pred_labels, class_names)

plot_loss_curves(resnet18_results), plot_confusion_matrix, cal_inference_time(resnet18, device, dummy_input=torch.rand(32, 3, 224, 224, dtype=torch.float))

In [ ]:
#Train ResNet50
resnet50.to(device)
rprop_optim = torch.optim.Rprop(params=resnet50.parameters(), lr=0.001)
resnet50_results = train(resnet50, train_dl, validation_dl, optimizer=rprop_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

In [ ]:
#Train VGG16
vgg16.to(device)
rprop_optim = torch.optim.Rprop(params=vgg16.parameters(), lr=0.001)
vgg16_results = train(vgg16, train_dl, validation_dl, optimizer=rprop_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

In [ ]:
#Train MobileNet Large
mobnet_lrg.to(device)
rprop_optim = torch.optim.Rprop(params=mobnet_lrg.parameters(), lr=0.001)
mobnet_lrg_results = train(mobnet_lrg, train_dl, validation_dl, optimizer=rprop_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

In [ ]:
#Train MobileNet Small
mobnet_sml.to(device)
rprop_optim = torch.optim.Rprop(params=mobnet_sml.parameters(), lr=0.001)
mobnet_sml_results = train(mobnet_sml, train_dl, validation_dl, optimizer=rprop_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

In [ ]:
#Train EfficientNet ...